In [30]:
import numpy as np
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)

# Lennard-Jones Potential
$V_{\text{LJ}}(r) = 4\epsilon \left[ \left( \frac{\sigma}{r} \right)^{12} - \left( \frac{\sigma}{r} \right)^{6} \right] $





In [31]:
# Function to calculate Lennard-Jones potential
def lj_potential(r2, sigma, epsilon):
    r2_inv = (sigma ** 2) / r2
    r6_inv = r2_inv ** 3
    r12_inv = r6_inv ** 2
    return 4 * epsilon * (r12_inv - r6_inv)

In [32]:
# Function to calculate the total potential energy of the system
def total_energy(positions, num_particles, box_length, sigma, epsilon, r_cut):
    energy = 0.0
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            rij = positions[i] - positions[j]
            rij = rij - box_length * np.rint(rij / box_length)  # Periodic boundary conditions
            r2 = np.dot(rij, rij)
            if r2 < r_cut ** 2:
                energy += lj_potential(r2, sigma, epsilon)
    return energy

In [33]:
# Read initial positions from XYZ file
def read_xyz(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        num_particles = int(lines[0])
        atom_data = [line.split() for line in lines[2:2 + num_particles]]
        atom_types = [data[0] for data in atom_data]
        positions = np.array([[float(x) for x in data[1:4]] for data in atom_data])
    return atom_types, positions

In [34]:
# Write positions to XYZ file
def write_xyz(file, atom_types, positions, step, energy):
    num_particles = len(atom_types)
    with open(file, 'a') as f:
        f.write(f"{num_particles}\n")
        f.write(f"Step {step}, Energy = {energy}\n")
        for atom_type, pos in zip(atom_types, positions):
            f.write(f"{atom_type} {pos[0]:.6f} {pos[1]:.6f} {pos[2]:.6f}\n")


In [35]:
# Lennard-Jones potential parameters for Argon
sigma = 3.4  # in Angstroms
epsilon = 0.238  # in kcal/mol

# Simulation parameters
temperature = 200.0  # Temperature in Kelvin
r_cut = 2.5 * sigma  # Cutoff distance, scaled by sigma
num_steps = 10000  # Number of Monte Carlo steps
dr_max = 0.1 * sigma  # Maximum displacement, scaled by sigma
box_length = 10.0  # Box dimension in Angstroms

In [36]:
input_file = 'argons.xyz'
output_file = 'trajectory_out.xyz'
atom_types, positions = read_xyz(input_file)
num_particles = len(atom_types)

In [37]:
for step in range(1, num_steps + 1):
    i = np.random.randint(num_particles)
    old_position = positions[i].copy()
    old_energy = total_energy(positions, num_particles, box_length, sigma, epsilon, r_cut)

    # Trial move
    positions[i] += dr_max * (np.random.rand(3) - 0.5)
    positions[i] = positions[i] - box_length * np.rint(positions[i] / box_length)  # PBC

    new_energy = total_energy(positions, num_particles, box_length, sigma, epsilon, r_cut)
    delta_energy = new_energy - old_energy

    # Metropolis criterion
    if np.random.rand() < np.exp(-delta_energy / (1.38064852e-23 * temperature / 1.9872041e-3)):
        energy = new_energy
    else:
        positions[i] = old_position  # Reject move

    # Write positions to the output file after each step
    if step%10==0:
        write_xyz(output_file, atom_types, positions, step, energy)